In [44]:
import pickle
import pandas as pd

In [45]:
# load the pickle file
with open('PrimaryReservesD-2-2023_ny.pkl', 'rb') as f:
    D2 = pickle.load(f)

# load the pickle file
with open('PrimaryReservesD-1-2023_ny.pkl', 'rb') as f:
    D1 = pickle.load(f)

# load picke 2023_spot

with open('2023_spot_norway.pkl', 'rb') as f:
    spot = pickle.load(f)

In [46]:
# merge D2['NO1'] with first column of spot
# pd.merge(date, NO5, how='left', left_on='Datetime', right_on='Datetime')

areas = {}

for i, key in enumerate(D2.keys()):
    areas[key] = pd.merge(spot.iloc[:,i], D1[key], how='left', right_on="Datetime", left_index=True)
    areas[key] = pd.merge(areas[key], D2[key], how='left', right_on="Datetime", left_on="Datetime")
    


In [47]:
# Set up price and volume for FFR Profil
areas['NO1']['FFR Profil EUR/MW'] = 0
areas['NO1']['FFR Profil MW'] = 0

# Start and end date for FFR Profil
date_start = '2023-05-22'
date_end = '2023-09-02'

# Start and end time for FFR Profil
time_start = 22
time_end = 7

# Logic for FFR Profil (Always active on weekends)
FFR_Profil_daterange = (areas['NO1']['Datetime'] >= date_start) & (areas['NO1']['Datetime'] <= date_end) & ((areas['NO1']['Datetime'].dt.weekday >= 5) | ((areas['NO1']['Datetime'].dt.hour >= time_start) | (areas['NO1']['Datetime'].dt.hour < time_end)))

# Set price and volume for FFR Profil
NOK_TO_EUR = 1/11.31

for key in areas.keys():
    areas[key]['FFR Profil EUR/MW'] = 0
    areas[key]['FFR Profil MW'] = 0
    areas[key].loc[FFR_Profil_daterange, 'FFR Profil EUR/MW'] = 150 * NOK_TO_EUR
    areas[key].loc[FFR_Profil_daterange, 'FFR Profil MW'] = 50

""" 
areas['NO1'].loc[FFR_Profil_daterange, 'FFR Profil EUR/MW'] = 150 * NOK_TO_EUR
areas['NO1'].loc[FFR_Profil_daterange, 'FFR Profil MW'] = 50
 """


/var/folders/3c/tpkrzdm9581crcp362vng33m0000gn/T/ipykernel_1303/1854518242.py:22: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '13.26259946949602' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  areas[key].loc[FFR_Profil_daterange, 'FFR Profil EUR/MW'] = 150 * NOK_TO_EUR
/var/folders/3c/tpkrzdm9581crcp362vng33m0000gn/T/ipykernel_1303/1854518242.py:22: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '13.26259946949602' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  areas[key].loc[FFR_Profil_daterange, 'FFR Profil EUR/MW'] = 150 * NOK_TO_EUR
/var/folders/3c/tpkrzdm9581crcp362vng33m0000gn/T/ipykernel_1303/1854518242.py:22: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Valu

" \nareas['NO1'].loc[FFR_Profil_daterange, 'FFR Profil EUR/MW'] = 150 * NOK_TO_EUR\nareas['NO1'].loc[FFR_Profil_daterange, 'FFR Profil MW'] = 50\n "

In [48]:
# Set up price and volume for FFR Flex

areas['NO1']['FFR Flex EUR/MW'] = 0
areas['NO1']['FFR Flex MW'] = 0

date_start = '2023-04-09'
date_end = '2023-10-27'

time_start = 8
time_end = 12

FFR_Flex_daterange = (areas['NO1']['Datetime'] >= date_start) & (areas['NO1']['Datetime'] <= date_end) & ( ((areas['NO1']['Datetime'].dt.hour >= time_start) & (areas['NO1']['Datetime'].dt.hour < time_end)))

for key in areas.keys():
    areas[key]['FFR Flex EUR/MW'] = 0
    areas[key]['FFR Flex MW'] = 0
    areas[key].loc[FFR_Flex_daterange, 'FFR Flex EUR/MW'] = 450 * NOK_TO_EUR
    areas[key].loc[FFR_Flex_daterange, 'FFR Flex MW'] = 102



""" 
areas['NO1'].loc[FFR_Flex_daterange, 'FFR Flex EUR/MW'] = 450 * NOK_TO_EUR

areas['NO1'].loc[FFR_Flex_daterange, 'FFR Flex MW'] = 102 
"""





/var/folders/3c/tpkrzdm9581crcp362vng33m0000gn/T/ipykernel_1303/360272971.py:17: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '39.78779840848806' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  areas[key].loc[FFR_Flex_daterange, 'FFR Flex EUR/MW'] = 450 * NOK_TO_EUR
/var/folders/3c/tpkrzdm9581crcp362vng33m0000gn/T/ipykernel_1303/360272971.py:17: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '39.78779840848806' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  areas[key].loc[FFR_Flex_daterange, 'FFR Flex EUR/MW'] = 450 * NOK_TO_EUR
/var/folders/3c/tpkrzdm9581crcp362vng33m0000gn/T/ipykernel_1303/360272971.py:17: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '39.78779

" \nareas['NO1'].loc[FFR_Flex_daterange, 'FFR Flex EUR/MW'] = 450 * NOK_TO_EUR\n\nareas['NO1'].loc[FFR_Flex_daterange, 'FFR Flex MW'] = 102 \n"

In [49]:
# Set up price and volume for aFFR opp
from entsoe import EntsoePandasClient
import pandas as pd

client = EntsoePandasClient(api_key='c008ebef-0f0f-42b2-a2ae-7330192177ac')

start = pd.Timestamp('20230101', tz='Europe/Brussels')
end = pd.Timestamp('20231231', tz='Europe/Brussels')
country_code = 'NO_1'  # Belgium
type_marketagreement_type = 'A01'

# get the data
#day_ahead = client.query_day_ahead_prices(country_code, start=start, end=end) #KeyError: '60T'
reservemarked_pris = client.query_contracted_reserve_prices(country_code, type_marketagreement_type, start=start, end=end, psr_type=None)
reservemarked_volum = client.query_contracted_reserve_amount(country_code, type_marketagreement_type, start=start, end=end, psr_type=None)

#clean the data
reservemarked_pris.columns = reservemarked_pris.columns.droplevel(0) #remove multiindex in columns
reservemarked_pris = reservemarked_pris.drop(columns=['Symmetric']) #del 'Frequency containment reserve' column
reservemarked_pris = reservemarked_pris.rename(columns={'Down': 'aFRR down EUR/MW'}) #rename columnns
reservemarked_pris = reservemarked_pris.rename(columns={'Up': 'aFRR up EUR/MW'}) #rename columnns

reservemarked_volum.columns = reservemarked_volum.columns.droplevel(0) #remove multiindex in columns
reservemarked_volum = reservemarked_volum.drop(columns=['Symmetric']) #del 'Frequency containment reserve' column
reservemarked_volum = reservemarked_volum.rename(columns={'Down': 'aFRR down MW'}) #rename columnns
reservemarked_volum = reservemarked_volum.rename(columns={'Up': 'aFRR up MW'}) #rename columnns


reservemarked_pris = reservemarked_pris.reset_index() #remove index and set it as a column
reservemarked_pris = reservemarked_pris.rename(columns={'index': 'Datetime'}) #change name of column
#reservemarked_pris['Datetime'] = reservemarked_pris['Datetime'].dt.tz_convert('Europe/Brussels') #convert to UTC
reservemarked_pris['Datetime'] = reservemarked_pris['Datetime'].dt.tz_convert('UTC') #convert to UTC
reservemarked_pris['Datetime'] = reservemarked_pris['Datetime'] + pd.Timedelta(hours=1)


reservemarked_volum = reservemarked_volum.reset_index() #remove index and set it as a column
reservemarked_volum = reservemarked_volum.rename(columns={'index': 'Datetime'}) #change name of column
#reservemarked_volum['Datetime'] = reservemarked_volum['Datetime'].dt.tz_convert('Europe/Brussels') #convert to UTC
reservemarked_volum['Datetime'] = reservemarked_volum['Datetime'].dt.tz_convert('UTC') #convert to UTC
reservemarked_volum['Datetime'] = reservemarked_volum['Datetime'] + pd.Timedelta(hours=1)


#merge the data into areas['NO1']
areas['NO1'] = pd.merge(areas['NO1'], reservemarked_pris, how='left', right_on="Datetime", left_on="Datetime")
areas['NO1'] = pd.merge(areas['NO1'], reservemarked_volum, how='left', right_on="Datetime", left_on="Datetime")



/Users/mathiasmollatt/opt/miniconda3/envs/inf200june/lib/python3.11/site-packages/entsoe/parsers.py:449: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  df = df.groupby(axis=1, level = [0,1]).mean()
/Users/mathiasmollatt/opt/miniconda3/envs/inf200june/lib/python3.11/site-packages/entsoe/parsers.py:449: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  df = df.groupby(axis=1, level = [0,1]).mean()
/Users/mathiasmollatt/opt/miniconda3/envs/inf200june/lib/python3.11/site-packages/entsoe/parsers.py:449: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  df = df.groupby(axis=1, level = [0,1]).mean()
/Users/mathiasmollatt/opt/miniconda3/envs/inf200june/lib/python3.11/site-packages/entsoe/parsers.py:449: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  df

In [51]:
    # Load the CSV file into a DataFrame
    df = pd.read_csv('Accepted Offers and Activated Balancing Reserves_202301010000-202401010000.csv')

    # Filter the DataFrame to include only rows where 'Source' is 'Not specified'
    filtered_df = df[df['Source'] == 'Not specified']

    # Rename columns for clarity
    renamed_df = filtered_df.rename(columns={
        'Balancing Time Unit (Automatic Frequency Restoration Reserve (aFRR))': 'Datetime',
        'Regulation Up Accepted [MW] - MBA|NO1': 'aFRR up MW v2',
        'Regulation Up Activated [MWh] - MBA|NO1': 'aFRR up MWh activated',
        'Regulation Down Accepted [MW] - MBA|NO1': 'aFRR down MW v2',
        'Regulation Down Activated [MWh] - MBA|NO1': 'aFRR down MWh activated'
    })

    # Preprocess 'Datetime' to extract just the start datetime and convert to pandas datetime objects
    # Assuming the format is 'start - end' and the datetime format is 'DD.MM.YYYY HH:MM'
    renamed_df['Datetime'] = pd.to_datetime(renamed_df['Datetime'].apply(lambda x: x.split(' - ')[0]), format='%d.%m.%Y %H:%M')

    # Make 'Datetime' timezone-aware, assuming it's in UTC
    renamed_df['Datetime'] = renamed_df['Datetime'].dt.tz_localize('UTC')
    #renamed_df['Datetime'] = renamed_df['Datetime'].dt.tz_localize(None)
    #convert to UTC
    #renamed_df['Datetime'] = renamed_df['Datetime'].dt.tz_convert('Europe/Brussels')

    final_df = renamed_df.drop(columns=['Source'])

    # Force columns to float, replacing commas with dots
    float_columns = ['aFRR up MW v2', 'aFRR up MWh activated', 'aFRR down MW v2', 'aFRR down MWh activated']
    for col in float_columns:
        final_df[col] = final_df[col].str.replace(',', '.').astype(float)


    # Merge the data into areas['NO1']
    areas['NO1'] = pd.merge(areas['NO1'], final_df, how='left', right_on="Datetime", left_on="Datetime")


In [61]:

# Load the dataset
file_path = 'Prices of Activated Balancing Energy_202301010000-202401010000.csv'
data = pd.read_csv(file_path)

# Modify the 'Time Interval' column to only keep the start datetime
data['Time Interval'] = pd.to_datetime(data['Time Interval'].apply(lambda x: x.split(' - ')[0]), format='%d.%m.%Y %H:%M')

# Keep only Time Interval, Up price, Down price
data = data[['Time Interval', 'Up price', 'Down Price']]

# Rename columns for clarity
data = data.rename(columns={
    'Time Interval': 'Datetime',
    'Up price': 'mFRR EAM up EUR/MWh',
    'Down Price': 'mFRR EAM down EUR/MWh'
})

# Make 'Datetime' timezone-aware, assuming it's in UTC
#renamed_df['Datetime'] = renamed_df['Datetime'].dt.tz_localize('UTC')
#renamed_df['Datetime'] = renamed_df['Datetime'].dt.tz_localize(None)
#convert to UTC
renamed_df['Datetime'] = renamed_df['Datetime'].dt.tz_convert('UTC')



# Merge the data into areas['NO1']
areas['NO1'] = pd.merge(areas['NO1'], data, how='left', right_on="Datetime", left_on="Datetime")


In [62]:
# dump the pickle file
with open('2023_norway.pkl', 'wb') as f:
    pickle.dump(areas, f)

In [63]:
# make to excel, index = Datetime
# datetime has to be utc removed

for key in areas.keys():
    areas[key]['Datetime'] = areas[key]['Datetime'].dt.tz_localize(None)
    

    areas[key].to_excel('excel/'+ key + '.xlsx', index=False)

In [64]:
areas['NO1']


,NO1 EUR_per_MWh,Datetime,D-1 FCR N EUR/MW,D-1 FCR N MW,D-1 FCR D up EUR/MW,D-1 FCR D up MW,D-2 FCR N EUR/MW,D-2 FCR N MW,D-2 FCR D up EUR/MW,D-2 FCR D up MW,...,aFRR down EUR/MW,aFRR up EUR/MW,aFRR down MW,aFRR up MW,aFRR up MW v2,aFRR up MWh activated,aFRR down MW v2,aFRR down MWh activated,mFRR EAM up EUR/MWh,mFRR EAM down EUR/MWh
0,119.32,2023-01-01 00:00:00,25.00,10.0,-99.0,0.0,63.0,15.0,-99.0,0.0,...,104.0,80.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
1,108.83,2023-01-01 01:00:00,35.80,10.0,-99.0,0.0,64.9,15.0,-99.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,108.83,40.00
2,102.39,2023-01-01 02:00:00,52.40,8.0,-99.0,0.0,64.9,15.0,-99.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,102.39,30.00
3,92.36,2023-01-01 03:00:00,71.50,8.0,-99.0,0.0,64.9,15.0,-99.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,92.36,-2.00
4,82.66,2023-01-01 04:00:00,90.00,8.0,-99.0,0.0,64.9,15.0,-99.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,82.66,-2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8758,62.88,2023-12-31 19:00:00,22.24,20.0,-99.0,0.0,20.0,18.0,-99.0,0.0,...,NaN,NaN,NaN,NaN,0.0,0.0,75.0,49.0,63.00,62.88
8759,62.60,2023-12-31 20:00:00,23.10,20.0,-99.0,0.0,20.0,18.0,-99.0,0.0,...,NaN,NaN,NaN,NaN,3.0,1.0,75.0,18.0,62.60,62.60
8760,62.25,2023-12-31 21:00:00,23.10,21.0,-99.0,0.0,20.0,18.0,-99.0,0.0,...,NaN,NaN,NaN,NaN,0.0,0.0,75.0,3.0,62.25,62.25
8761,61.77,2023-12-31 22:00:00,22.00,21.0,-99.0,0.0,20.0,18.0,-99.0,0.0,...,NaN,NaN,NaN,NaN,0.0,0.0,69.0,36.0,61.77,61.77
